In [128]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
orders = pd.read_csv('data/orders.csv')
cc = pd.read_csv('data/credit_cards.csv')
ba = pd.read_csv('data/bank_accounts.csv')
device = pd.read_csv('data/devices.csv')

C:\Users\tommy.yong\AppData\Local\Continuum\anaconda3\envs\learnenv\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [20]:
cc_ba = cc.join(ba, on='userid', lsuffix='_cc', rsuffix='_ba', how='outer')
cc_ba['userid'] = cc_ba['userid_cc'].combine_first(cc_ba['userid_ba'])
cc_ba.drop(['userid_cc', 'userid_ba'], axis=1, inplace=True)

In [68]:
cc_ba_device = cc_ba.join(device, on='userid', lsuffix='_l', rsuffix='_r', how='outer')
cc_ba_device['userid'] = cc_ba_device['userid_l'].combine_first(cc_ba_device['userid_r'])
cc_ba_device.drop(['userid_l', 'userid_r'], axis=1, inplace=True)

In [36]:
# cc_ba_device[(cc_ba_device['credit_card'].isna() == False) & (cc_ba_device['bank_account'].isna() == False) & (cc_ba_device['device'].isna() == False)]

In [69]:
cc_cnt = cc_ba_device.groupby('credit_card').count().iloc[:, 0]
cc_cnt = cc_cnt[cc_cnt > 1]
ba_cnt = cc_ba_device.groupby('bank_account').count().iloc[:, 0]
ba_cnt = ba_cnt[ba_cnt > 1]
device_cnt = cc_ba_device.groupby('device').count().iloc[:, 0]
device_cnt = device_cnt[device_cnt > 1]

In [70]:
potential_fraud_users = cc_ba_device[cc_ba_device['credit_card'].apply(lambda x: x in cc_cnt.index) \
                            | cc_ba_device['bank_account'].apply(lambda x: x in ba_cnt.index) \
                           | cc_ba_device['device'].apply(lambda x: x in device_cnt.index)]

In [78]:
pfl = potential_fraud_users['userid'].unique()

In [80]:
orders_filter = orders[orders['buyer_userid'].apply(lambda x: x in pfl) \
                      & orders['seller_userid'].apply(lambda x: x in pfl)]

In [81]:
orders_filter

,orderid,buyer_userid,seller_userid
3,1953314712,131221930,72837912
9,1953480060,130236220,77145726
13,1953537154,210510748,34245056
21,1953567064,158115678,103582530
31,1953702496,175905100,55708090
39,1953793440,45479694,90994546
41,1953805742,43011050,121514782
42,1953838500,159382020,38091514
49,1953930216,164555846,16774176
57,1953997418,1382076,6628642


In [82]:
len(orders)

620947

In [85]:
orders.join(orders_filter, how='inner', on='orderid', lsuffix='_l', rsuffix='_r')

,orderid,orderid_l,buyer_userid_l,seller_userid_l,orderid_r,buyer_userid_r,seller_userid_r


In [91]:
orders.iloc[orders_filter.index]['is_fraud'] = 'not fraud'

C:\Users\tommy.yong\AppData\Local\Continuum\anaconda3\envs\learnenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:

for idx in list(orders_filter.index):
    

False

In [179]:
orders['is_fraud'] = 'not fraud'

In [119]:
orders.loc[orders_filter.index]['is_fraud'] = True

In [175]:
orders.loc[orders_filter.index, ['is_fraud']] = 'Treu'

In [180]:
orders[['orderid', 'is_fraud']].to_csv('submission.csv', index=False)

In [129]:
engine = create_engine('sqlite:///test.db')

In [131]:
orders_filter.to_sql('odf', con=engine, if_exists='replace')
potential_fraud_users.to_sql('pfu', con=engine, if_exists='replace')

In [151]:
stm = '''
select buyer_bank_account, bank_account from 
    (select 
        *,
        case 
            when buyer_credit_card = credit_card then "same"
            else "notsame"
        end as same_cc,
        case 
            when buyer_bank_account = bank_account then "same"
            else "notsame"
        end as same_ba,
        case 
            when buyer_device = device then "same"
            else "notsame"
        end as same_device
    from (select * from (
            select 
                orderid, 
                buyer_userid, 
                credit_card as buyer_credit_card, 
                bank_account as buyer_bank_account,
                device as buyer_device,
                seller_userid
            from odf
            left join pfu
            on odf.buyer_userid = pfu.userid
            where buyer_credit_card is not null
        ) as a
        left join pfu
        on a.seller_userid = pfu.userid
        where credit_card is not null) as aa) as aaa
'''

df = pd.read_sql_query(stm, con=engine)

In [169]:
# df[(df['buyer_credit_card'] == df['credit_card'])]
df[(df['bank_account'] == None)]

,buyer_bank_account,bank_account


In [178]:
df['bank_account']

0      None
1      None
2      None
3      None
4      None
       ... 
233    None
234    None
235    None
236    None
237    None
Name: bank_account, Length: 238, dtype: object

In [181]:
orders_filter

,orderid,buyer_userid,seller_userid
3,1953314712,131221930,72837912
9,1953480060,130236220,77145726
13,1953537154,210510748,34245056
21,1953567064,158115678,103582530
31,1953702496,175905100,55708090
39,1953793440,45479694,90994546
41,1953805742,43011050,121514782
42,1953838500,159382020,38091514
49,1953930216,164555846,16774176
57,1953997418,1382076,6628642
